# Laboratorio 7 - HMM y CRF con dataset real de NER en español
Francisco Castillo - 21562

## Ejercicio 1 - HMM:

### a) Instala e importa el dataset

In [24]:
from datasets import load_dataset
dataset = load_dataset("IEETA/SPACCC-Spanish-NER")

### b) Explora el dataset

In [25]:
train, test = dataset["train"], dataset["test"]
print(f"Train size: {len(train)}")
print(f"Test size: {len(test)}")

Train size: 33757
Test size: 11239


In [26]:
from pprint import pprint

pprint(train[:5])

{'ann_id': [0, 1, 2, 3, 4],
 'end_span': [73, 190, 278, 230, 305],
 'filename': ['S0004-06142005000500011-1',
              'S0004-06142005000500011-1',
              'S0004-06142005000500011-1',
              'S0004-06142005000500011-1',
              'S0004-06142005000500011-1'],
 'label': ['DISEASE', 'DISEASE', 'PROCEDURE', 'DISEASE', 'DISEASE'],
 'start_span': [50, 158, 192, 207, 280],
 'text': ['alergias medicamentosas',
          'fracturas vertebrales y costales',
          'intervenido de enfermedad de Dupuytren en mano derecha y by-pass '
          'iliofemoral izquierdo',
          'enfermedad de Dupuytren',
          'Diabetes Mellitus tipo II']}


El dataset no contiene etiquetas BIO predeterminadas, por lo que procedemos a crearlas manualmente. Es de notar que e `text` no se encuentra dentro de un corpus de texto, por lo que no hay palabras que no pertenezcan a una entidad nombrada; es decir, todas las palabras deben ser etiquetadas como parte de una entidad nombrada (B-XXX o I-XXX).

### c) Preprocesamiento

#### Creación de etiquetas BIO

In [27]:
def create_bio_labels(text, label):
    return [f"B-{label}"] + [f"I-{label}"] * (len(text.split()) - 1)

In [28]:
bio_tags = set()
for split in dataset.keys():
    for example in dataset[split]:
        bio_tags.update(create_bio_labels(example["text"], example["label"]))
bio_tags = sorted(bio_tags)  # sorted list of unique BIO tags
bio_tags = {tag: idx for idx, tag in enumerate(bio_tags)}  # mapping to integers

In [29]:
bio_tags

{'B-CHEMICAL': 0,
 'B-DISEASE': 1,
 'B-PROCEDURE': 2,
 'B-PROTEIN': 3,
 'B-SYMPTOM': 4,
 'I-CHEMICAL': 5,
 'I-DISEASE': 6,
 'I-PROCEDURE': 7,
 'I-PROTEIN': 8,
 'I-SYMPTOM': 9}

#### Mapear cada token y etiqueta a un número entero

In [30]:
all_words = list(train) + list(test)
all_word =  sorted(set(
    word
    for example in all_words
    for word in example["text"].split()
))
word2idx = {word: idx for idx, word in enumerate(sorted(all_word))}

In [31]:
def get_numbers(observation):
    text = observation["text"]
    label = observation["label"]
    return{
        "tokens": [word2idx[word] for word in text.split()],
        "bio_labels": [bio_tags[tag] for tag in create_bio_labels(text, label)]
    }

In [32]:
train = train.map(lambda x: get_numbers(x))
test = test.map(lambda x: get_numbers(x))

In [33]:
train[0]

{'filename': 'S0004-06142005000500011-1',
 'ann_id': 0,
 'label': 'DISEASE',
 'start_span': 50,
 'end_span': 73,
 'text': 'alergias medicamentosas',
 'tokens': [4195, 12111],
 'bio_labels': [1, 6]}

### d) Calculo de Marices

In [34]:
import numpy as np

n_tags = len(bio_tags)
n_words = len(word2idx)

# Initialize matrices
transition_counts = np.zeros((n_tags, n_tags), dtype=int)
emission_counts = np.zeros((n_tags, n_words), dtype=int)

# Count transitions and emissions
for example in train:
    labels = example["bio_labels"]
    tokens = example["tokens"]
    for i in range(len(labels)):
        emission_counts[labels[i], tokens[i]] += 1
        if i > 0:
            transition_counts[labels[i-1], labels[i]] += 1

# Normalize to get probabilities
transition_matrix = transition_counts / transition_counts.sum(axis=1, keepdims=True)
emission_matrix = emission_counts / emission_counts.sum(axis=1, keepdims=True)

In [35]:
# Handle NaN values (rows that sum to zero)
transition_matrix = np.nan_to_num(transition_matrix)
emission_matrix = np.nan_to_num(emission_matrix)

In [36]:
transition_matrix

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [37]:
emission_matrix.shape

(10, 17838)

### e) HMM Multinomial

In [38]:
import numpy as np
from hmmlearn.hmm import CategoricalHMM

In [39]:
idx2tag = {v: k for k, v in bio_tags.items()}
idx2word = {v: k for k, v in word2idx.items()}

In [40]:
# Helper: pack sequences into hmmlearn's expected shape
def pack_sequences(ds, key="tokens"):
    lengths = [len(ex[key]) for ex in ds]
    X = np.concatenate([np.asarray(ex[key], dtype=int).reshape(-1, 1) for ex in ds], axis=0)
    return X, lengths

In [41]:
# Compute start probabilities from first labels in each sequence
n_tags = len(bio_tags)
n_words = len(word2idx)
start_counts = np.zeros(n_tags, dtype=float)
for ex in train:
    if len(ex["bio_labels"]) > 0:
        start_counts[ex["bio_labels"][0]] += 1.0

In [42]:
# Laplace smoothing and normalization
eps = 1e-8
startprob = start_counts + eps
startprob = startprob / startprob.sum()

# (transition_counts: [n_tags, n_tags], emission_counts: [n_tags, n_words])
transmat = transition_counts.astype(float) + eps
transmat = transmat / np.clip(transmat.sum(axis=1, keepdims=True), 1.0, None)

emissionprob = emission_counts.astype(float) + eps
emissionprob = emissionprob / np.clip(emissionprob.sum(axis=1, keepdims=True), 1.0, None)

In [43]:
X_train, lengths_train = pack_sequences(train)

model = CategoricalHMM(
    n_components=n_tags,
    init_params="",  # do not overwrite our params
    params="",       # do not update during fit
    random_state=0
)
model.n_features = n_words
model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob

### f) Decodificación con Viterbi

In [44]:
all_correct = 0
all_total = 0
for ex in test:
    obs = np.asarray(ex["tokens"], dtype=int).reshape(-1, 1)
    y_pred = model.predict(obs)
    y_true = np.asarray(ex["bio_labels"], dtype=int)
    all_correct += (y_pred == y_true).sum()
    all_total += len(y_true)

print("Token accuracy (test set): {:.3f}".format(all_correct / max(all_total, 1)))

Token accuracy (test set): 0.811


### Reflexiona
#### ¿Cómo afecta la diversidad de datos a las matrices?

La diversidad de los datos influye directamente en las matrices de transición y emisión del HMM. Si algunas palabras o etiquetas aparecen muy pocas veces, las probabilidades estimadas para esas combinaciones serán muy bajas o incluso cero, lo que puede llevar al modelo a favorecer etiquetas frecuentes y a ignorar secuencias poco representadas. Además, la diversidad limitada puede causar que ciertas transiciones nunca se observen en el entrenamiento, generando huecos en la matriz de transición que afectan la capacidad del HMM para generalizar a nuevos ejemplos.

En este caso, hemos mencionado que no existen etiquetas `O` en el dataset, lo que significa que todas las palabras deben ser etiquetadas como parte de una entidad nombrada. Esto puede llevar a que el modelo tenga dificultades para manejar palabras que no pertenecen a ninguna entidad en un contexto real, ya que no ha aprendido a reconocerlas. La falta de diversidad en las etiquetas puede limitar la capacidad del modelo para adaptarse a situaciones del mundo real donde no todas las palabras son parte de entidades nombradas.

---

#### ¿Qué etiquetas o palabras presentan probabilidades bajas o cero y cómo podría resolverse?

Las palabras raras o únicas en el dataset, así como etiquetas poco frecuentes, tienden a tener emisiones o transiciones con probabilidades muy bajas o cero. Esto incluye nombres propios poco comunes, errores tipográficos o entidades que aparecen solo en unos pocos ejemplos. Además, palabras que aparecen en el test pero no en el entrenamiento (`OOV`) no tendrán representación en la matriz de emisión, lo que puede generar predicciones incorrectas o ceros. Esto, pues creamos un token para todas las palabras en el dataset (tanto train como test).

Para resolverlo, se puede aplicar Laplace Smoothing para evitar probabilidades nulas, introducir un token `<UNK>` para palabras desconocidas, agrupar etiquetas muy poco frecuentes en categorías generales y aumentar ejemplos de entidades raras. Además de, obviamente, incluir palabras con la etiqueta `O` en el dataset para que el modelo pueda aprender a manejarlas.



## Ejercicio 2 - CRF: Entrenamiento con SPACCC

### a) Define features para cada token:
Minúsculas, mayúsculas, prefijos y sufijos.
Palabra anterior y siguiente.
Presencia de dígitos o caracteres especiales.

In [45]:
from sklearn_crfsuite import CRF, metrics

In [46]:
def token_features(tokens, i):
    word = tokens[i]
    feats = {
        "bias": 1.0,
        "word.lower": word.lower(),
        "word.isupper": word.isupper(),
        "word.istitle": word.istitle(),
        "word.isdigit": word.isdigit(),
        "prefix1": word[:1].lower(),
        "prefix2": word[:2].lower(),
        "prefix3": word[:3].lower(),
        "suffix1": word[-1:].lower(),
        "suffix2": word[-2:].lower() if len(word) >= 2 else word.lower(),
        "suffix3": word[-3:].lower() if len(word) >= 3 else word.lower(),
        "has_digit": any(ch.isdigit() for ch in word),
        "has_punct": any(not ch.isalnum() for ch in word),
        "has_hyphen": "-" in word,
    }
    if i == 0:
        feats["BOS"] = True
    else:
        prev = tokens[i - 1]
        feats.update({
            "-1:word.lower": prev.lower(),
            "-1:word.istitle": prev.istitle(),
            "-1:word.isupper": prev.isupper(),
        })
    if i == len(tokens) - 1:
        feats["EOS"] = True
    else:
        nxt = tokens[i + 1]
        feats.update({
            "+1:word.lower": nxt.lower(),
            "+1:word.istitle": nxt.istitle(),
            "+1:word.isupper": nxt.isupper(),
        })
    return feats

In [47]:
def example_to_features_and_labels(example):
    tokens = example["text"].split()
    X = [token_features(tokens, i) for i in range(len(tokens))]
    y = create_bio_labels(example["text"], example["label"])  # string BIO tags
    return X, y

In [48]:
X_train, y_train = [], []
for ex in train:
    X, y = example_to_features_and_labels(ex)
    X_train.append(X)
    y_train.append(y)

X_test, y_test = [], []
for ex in test:
    X, y = example_to_features_and_labels(ex)
    X_test.append(X)
    y_test.append(y)

In [49]:
crf = CRF(
    algorithm="lbfgs",
    c1=0.25,                 # L1 regularization
    c2=0.25,                 # L2 regularization
    max_iterations=200,
    all_possible_transitions=False,
)
crf.fit(X_train, y_train)

,algorithm,'lbfgs'
,min_freq,None
,all_possible_states,None
,all_possible_transitions,False
,c1,0.25
,c2,0.25
,max_iterations,200
,num_memories,None
,epsilon,None
,period,None
,delta,None


In [50]:
y_pred = crf.predict(X_test)

In [51]:
def token_accuracy(y_true, y_pred):
    correct = sum(t == p for yt, yp in zip(y_true, y_pred) for t, p in zip(yt, yp))
    total = sum(len(yt) for yt in y_true)
    return (correct / total) if total else 0.0

In [55]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def flatten(seqs):
    return [x for seq in seqs for x in seq]

labels = [tag for tag, _idx in sorted(bio_tags.items(), key=lambda kv: kv[1])]

y_true_tags_hmm, y_pred_tags_hmm = [], []
for ex in test:
    obs = np.asarray(ex["tokens"], dtype=int).reshape(-1, 1)
    y_true_idx = np.asarray(ex["bio_labels"], dtype=int)
    y_pred_idx = model.predict(obs)
    y_true_tags_hmm.append([idx2tag[i] for i in y_true_idx])
    y_pred_tags_hmm.append([idx2tag[i] for i in y_pred_idx])

prec_hmm, _, _, _ = precision_recall_fscore_support(
    flatten(y_true_tags_hmm), flatten(y_pred_tags_hmm),
    labels=labels, average=None, zero_division=0
)
prec_crf, _, _, _ = precision_recall_fscore_support(
    flatten(y_test), flatten(y_pred),
    labels=labels, average=None, zero_division=0
)

precision_by_model = {
    "HMM": {label: float(p) for label, p in zip(labels, prec_hmm)},
    "CRF": {label: float(p) for label, p in zip(labels, prec_crf)},
}

precision_by_model

{'HMM': {'B-CHEMICAL': 0.833011583011583,
  'B-DISEASE': 0.8198659834450138,
  'B-PROCEDURE': 0.7930451581743541,
  'B-PROTEIN': 0.919093851132686,
  'B-SYMPTOM': 0.8503239004432321,
  'I-CHEMICAL': 0.4176470588235294,
  'I-DISEASE': 0.71877994251038,
  'I-PROCEDURE': 0.8824110671936759,
  'I-PROTEIN': 0.6794425087108014,
  'I-SYMPTOM': 0.8207062050051422},
 'CRF': {'B-CHEMICAL': 0.88379705400982,
  'B-DISEASE': 0.852689010132502,
  'B-PROCEDURE': 0.9228215767634855,
  'B-PROTEIN': 0.9064171122994652,
  'B-SYMPTOM': 0.8617227979274611,
  'I-CHEMICAL': 0.7777777777777778,
  'I-DISEASE': 0.8102254571276407,
  'I-PROCEDURE': 0.9076604554865424,
  'I-PROTEIN': 0.8446215139442231,
  'I-SYMPTOM': 0.8444020760512657}}

## Ejercicio 3 - Comparación de modelos

In [56]:
weighted_prec_hmm = metrics.flat_precision_score(y_true_tags_hmm, y_pred_tags_hmm, average='weighted', labels=labels)
weighted_prec_crf = metrics.flat_precision_score(y_test, y_pred, average='weighted',
                                                  labels=labels)
print(f"Weighted Precision HMM: {weighted_prec_hmm:.3f}")
print(f"Weighted Precision CRF: {weighted_prec_crf:.3f}")

Weighted Precision HMM: 0.816
Weighted Precision CRF: 0.865


In [59]:
import pandas as pd

df = pd.DataFrame(precision_by_model).reindex(labels)

df["Difference (CRF - HMM)"] = df["CRF"] - df["HMM"]

df = (
    df.reset_index()
      .rename(columns={"index": "Tag", "HMM": "HMM precision", "CRF": "CRF precision"})
      .loc[:, ["Tag", "HMM precision", "CRF precision", "Difference (CRF - HMM)"]]
      .round(3)
)

df

,Tag,HMM precision,CRF precision,Difference (CRF - HMM)
0,B-CHEMICAL,0.833,0.884,0.051
1,B-DISEASE,0.820,0.853,0.033
2,B-PROCEDURE,0.793,0.923,0.130
3,B-PROTEIN,0.919,0.906,-0.013
4,B-SYMPTOM,0.850,0.862,0.011
5,I-CHEMICAL,0.418,0.778,0.360
6,I-DISEASE,0.719,0.810,0.091
7,I-PROCEDURE,0.882,0.908,0.025
8,I-PROTEIN,0.679,0.845,0.165
9,I-SYMPTOM,0.821,0.844,0.024


In [61]:
arr_crf = np.array([precision_by_model["CRF"][t] for t in labels], dtype=float)
arr_hmm = np.array([precision_by_model["HMM"][t] for t in labels], dtype=float)

wins_crf = int(np.sum(arr_crf > arr_hmm))
wins_hmm = int(np.sum(arr_hmm > arr_crf))

total = len(labels)

summary = {
    "CRF": {
        "wins": wins_crf,
        "losses": wins_hmm,
    },
    "HMM": {
        "wins": wins_hmm,
        "losses": wins_crf,
    },
}

winrate_df = (
    pd.DataFrame(summary)
      .T.loc[:, ["wins", "losses"]]
      .round(3)
)

winrate_df

,wins,losses
CRF,9,1
HMM,1,9


#### Reflexión
##### ¿Cuál modelo generaliza mejor con pocas secuencias?
En los datos se observa que el CRF supera al HMM en 9 de 10 etiquetas, con mejoras en etiquetas BIO como I-CHEMICAL (+0.36) e I-PROTEIN (+0.165). Esto indica que el CRF logra capturar mejor la estructura dentro de las secuencias, incluso cuando las etiquetas dependen de contexto previo (B-XX). El único caso donde el HMM fue superior es en B-PROTEIN (-0.013), una diferencia mínima
##### ¿Qué ventajas ofrece el CRF con respecto a la utilización de features?
El CRF muestra ventajas en etiquetas que requieren mayor información contextual. Por ejemplo, en I-CHEMICAL la diferencia es de +0.36, lo que evidencia cómo el modelo aprovecha features adicionales para distinguir mejor cuándo continuar una entidad química (probablemente por qué contienen palabras no tan comúnes en el resto de tags). Similarmente, en I-PROTEIN y I-DISEASE se observa un salto considerable (+0.165 y +0.091, respectivamente). Esto refleja que el CRF puede integrar múltiples características de entrada que el HMM no considera, lo que resulta en un mejor rendimiento en secuencias internas más ambiguas.
##### ¿En qué escenarios sería suficiente un HMM?
Los datos muestran que el HMM compite relativamente bien en etiquetas de inicio como B-PROTEIN (0.919 vs 0.906) y mantiene resultados aceptables en otras etiquetas de inicio, donde las dependencias contextuales son menos críticas. Esto sugiere que un HMM puede ser suficiente en tareas donde lo más importante es identificar el comienzo de entidades y el contexto no aporta tanta información extra. En problemas de dominio cerrado (solamente un tipo de tag) o cuando los recursos computacionales y de datos son limitados, un HMM seguiría siendo una opción válida y más ligera.